<a href="https://colab.research.google.com/github/soumik12345/point-cloud-segmentation/blob/inference/notebooks/train_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/soumik12345/point-cloud-segmentation
!pip install -qqq wandb ml_collections

Cloning into 'point-cloud-segmentation'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (734/734), done.
remote: Compressing objects: 100% (460/460), done.
remote: Total 734 (delta 430), reused 487 (delta 228), pack-reused 0
Receiving objects: 100% (734/734), 2.99 MiB | 8.94 MiB/s, done.
Resolving deltas: 100% (430/430), done.
     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 88 kB 9.6 MB/s 
     |████████████████████████████████| 180 kB 89.2 MB/s 
     |████████████████████████████████| 139 kB 81.3 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [1]:
import sys

sys.path.append("point-cloud-segmentation")

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import os

import wandb
import wandb.keras
from datetime import datetime

from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import mixed_precision

from point_seg import TFRecordLoader, ShapeNetCoreLoaderInMemory
from point_seg import models, utils

In [4]:
#@title Configs
#@markdown Get your `wandb_api_key` from https://wandb.ai/authorize.
wandb_api_key = "" #@param {type:"string"}
object_category = "Bag" #@param {type:"string"}
num_points = 1024 #@param {type:"integer"}
batch_size = 32 #@param {type:"integer"}
val_split = 0.2 #@param {type:"number"}
epochs = 60 #@param {type:"integer"}
initial_lr = 1e-3 #@param {type:"number"}
drop_every = 10 #@param {type:"integer"}
decay_factor = 0.5 #@param {type:"number"}

In [5]:
timestamp = datetime.utcnow().strftime("%y%m%d-%H%M%S")
strategy = utils.initialize_device()
batch_size = 32 * strategy.num_replicas_in_sync

In [6]:
wandb.init(
    project='pointnet_shapenet_core',
    name=f"{object_category}_{timestamp}",
    entity="pointnet",
)

wandb: Currently logged in as: pointnet (use `wandb login --relogin` to force relogin)


In [7]:
# Apply mixed-precision policy [OPTIONAL]
mixed_precision.set_global_policy("mixed_float16")
policy = mixed_precision.global_policy()

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [7]:
data_loader = ShapeNetCoreLoaderInMemory(
    object_category=object_category,
    n_sampled_points=num_points,
)
data_loader.load_data()
train_dataset, val_dataset = data_loader.get_datasets(
    val_split=val_split,
    batch_size=batch_size,
)

100%|██████████| 79/79 [00:00<00:00, 395.00it/s]


In [8]:
lr_scheduler = utils.StepDecay(initial_lr, drop_every, decay_factor)
lr_callback = callbacks.LearningRateScheduler(
    lambda epoch: lr_scheduler(epoch), verbose=True
)

# Tensorboard Callback
logs_dir = os.path.join(
    "logs", f"{object_category}_{timestamp}"
)
logs_dir = os.path.join(logs_dir)
tb_callback = callbacks.TensorBoard(log_dir=logs_dir)

# ModelCheckpoint Callback
checkpoint_path = os.path.join(
    "training_checkpoints",
    f"{object_category}_{timestamp}.h5",
)
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_best_only=True, save_weights_only=True,
)

callback_list = [
    tb_callback,
    checkpoint_callback,
    lr_callback,
    wandb.keras.WandbCallback()
]

In [9]:
with strategy.scope():
    optimizer = optimizers.Adam(learning_rate=initial_lr)
    _, y = next(iter(train_dataset))
    num_classes = y.shape[-1]
    model = models.get_shape_segmentation_model(num_points, num_classes)

model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

In [10]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callback_list,
)

Epoch 1/60

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
2/2 [==============================] - 8s 2s/step - loss: 12.9136 - accuracy: 0.1329 - val_loss: 82.6287 - val_accuracy: 0.0618
Epoch 2/60

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
2/2 [==============================] - 0s 290ms/step - loss: 10.7233 - accuracy: 0.2365 - val_loss: 60.4566 - val_accuracy: 0.0065
Epoch 3/60

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
2/2 [==============================] - 0s 299ms/step - loss: 8.0717 - accuracy: 0.3747 - val_loss: 9.0782 - val_accuracy: 0.1403
Epoch 4/60

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
2/2 [==============================] - 0s 282ms/step - loss: 6.8129 - accuracy: 0.4994 - val_loss: 5.3153 - val_accuracy: 0.5540
Epoch 5/60

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
2/2 [==============================] - 0s 289ms/step - loss: 5.8839 - accuracy: 0.6959 -